In [1]:
!pip install -q torchaudio
!pip install -q transformers
!pip install -q datasets
!pip install -q lang_trans
!pip install -q arabic_reshaper
!pip install -q python-bidi
!pip install -q pydub
!pip install -q soundfile
!pip install -q jiwer
!pip install -q PyArabic

     |████████████████████████████████| 5.3 MB 5.0 MB/s 
     |████████████████████████████████| 7.6 MB 77.6 MB/s 
     |████████████████████████████████| 163 kB 75.7 MB/s 
     |████████████████████████████████| 441 kB 5.2 MB/s 
     |████████████████████████████████| 212 kB 91.8 MB/s 
     |████████████████████████████████| 115 kB 87.0 MB/s 
     |████████████████████████████████| 127 kB 84.7 MB/s 
     |████████████████████████████████| 1.4 MB 5.1 MB/s 
     |████████████████████████████████| 2.2 MB 81.3 MB/s 
     |████████████████████████████████| 126 kB 5.0 MB/s 


In [2]:
import transformers
transformers.utils.move_cache()

Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [3]:
import tensorflow as tf

In [4]:
import os
import re
import sys
import json
import torch
import jiwer
import logging
import librosa
import datasets
import itertools
import torchaudio
import numpy as np
import transformers
import pandas as pd
from torch import nn
import seaborn as sns
import torch.nn as nn
from tqdm import tqdm
import soundfile as sf
import arabic_reshaper
from pyarabic import araby
from packaging import version
from pydub import AudioSegment
from dataclasses import asdict
import matplotlib.pyplot as plt
from pydub.utils import mediainfo
from argparse import ArgumentParser 
from collections import defaultdict
from torch.nn import functional as F
from contextlib import contextmanager
from bidi.algorithm import get_display
from lang_trans.arabic import buckwalter
from dataclasses import dataclass, field
from datasets import load_dataset, Dataset
from sklearn.metrics import accuracy_score
from typing import Any, Dict, List, Optional, Union
from sklearn.model_selection import train_test_split
from transformers import HfArgumentParser,TrainingArguments
from sklearn.metrics import classification_report, confusion_matrix
from transformers.trainer_utils import get_last_checkpoint, is_main_process
from transformers import is_apex_available,set_seed ,Trainer,Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2Model,Wav2Vec2PreTrainedModel
%matplotlib inline

# Helper classes

## CTC Trainer

In [5]:
class CTCTrainer(Trainer):
    def training_step(self, model, inputs):

        model.train()
        inputs = self._prepare_inputs(inputs)
        loss = self.compute_loss(model, inputs)

        if self.args.gradient_accumulation_steps > 1:
            loss = loss / self.args.gradient_accumulation_steps

        loss.backward()

        return loss.detach()

    def compute_loss(self, model, inputs, return_outputs=False):
        # labels = inputs.pop("labels").to('cuda')
        labels = inputs["labels"].to("cuda")
        outputs = model(**inputs)
        loss_fct = torch.nn.CrossEntropyLoss()
        loss = loss_fct(outputs["logits"], labels.argmax(-1).long())
        return (loss, outputs) if return_outputs else loss

## Processors

In [6]:
class CustomWav2Vec2Processor:
    def __init__(self, feature_extractor):
        self.feature_extractor = feature_extractor
        self.current_processor = self.feature_extractor

    def save_pretrained(self, save_directory):
        self.feature_extractor.save_pretrained(save_directory)

    @classmethod
    def from_pretrained(cls, pretrained_model_name_or_path, **kwargs):
        feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(
            pretrained_model_name_or_path,
            **kwargs,
        )
        return cls(feature_extractor=feature_extractor)

    def __call__(self, *args, **kwargs):
        return self.current_processor(*args, **kwargs)

    def pad(self, *args, **kwargs):
        return self.current_processor.pad(*args, **kwargs)

## Model Args

In [7]:
def list_field(default=None, metadata=None):
    return field(default_factory=lambda: default, metadata=metadata)

@dataclass
class ModelArguments:

    model_name_or_path: str = field(
        metadata={
            "help": "Path to pretrained model or model identifier from huggingface.co/models"
        }
    )
    cache_dir: Optional[str] = field(
        default=None,
        metadata={
            "help": "Where do you want to store the pretrained models downloaded from huggingface.co"
        },
    )
    freeze_feature_extractor: Optional[bool] = field(
        default=True,
        metadata={
            "help": "Whether to freeze the feature extractor layers of the model."
        },
    )
    attention_dropout: Optional[float] = field(
        default=0.1,
        metadata={"help": "The dropout ratio for the attention probabilities."},
    )
    activation_dropout: Optional[float] = field(
        default=0.1,
        metadata={
            "help": "The dropout ratio for activations inside the fully connected layer."
        },
    )
    hidden_dropout: Optional[float] = field(
        default=0.1,
        metadata={
            "help": "The dropout probabilitiy for all fully connected layers in the embeddings, encoder, and pooler."
        },
    )
    feat_proj_dropout: Optional[float] = field(
        default=0.1,
        metadata={
            "help": "The dropout probabilitiy for all 1D convolutional layers in feature extractor."
        },
    )
    mask_time_prob: Optional[float] = field(
        default=0.05,
        metadata={
            "help": "Propability of each feature vector along the time axis to be chosen as the start of the vector"
            "span to be masked. Approximately ``mask_time_prob * sequence_length // mask_time_length`` feature"
            "vectors will be masked along the time axis. This is only relevant if ``apply_spec_augment is True``."
        },
    )
    gradient_checkpointing: Optional[bool] = field(
        default=True,
        metadata={
            "help": "If True, use gradient checkpointing to save memory at the expense of slower backward pass."
        },
    )
    layerdrop: Optional[float] = field(
        default=0.0, metadata={"help": "The LayerDrop probability."}
    )


@dataclass
class DataTrainingArguments:

    dataset_config_name: Optional[str] = field(
        default=None,
        metadata={
            "help": "The configuration name of the dataset to use (via the datasets library)."
        },
    )
    train_split_name: Optional[str] = field(
        default="train+validation",
        metadata={
            "help": "The name of the training data set split to use (via the datasets library). Defaults to 'train'"
        },
    )
    overwrite_cache: bool = field(
        default=False,
        metadata={"help": "Overwrite the cached preprocessed datasets or not."},
    )
    preprocessing_num_workers: Optional[int] = field(
        default=None,
        metadata={"help": "The number of processes to use for the preprocessing."},
    )
    max_train_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of training examples to this "
            "value if set."
        },
    )
    max_val_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of validation examples to this "
            "value if set."
        },
    )
    chars_to_ignore: List[str] = list_field(
        default=[",", "?", ".", "!", "-", ";", ":", '""', "%", "'", '"', "�"],
        metadata={"help": "A list of characters to remove from the transcripts."},
    )


@dataclass
class DataCollatorCTCWithPadding:

    processor: CustomWav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = 320000
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = 320000
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods

        input_features = [
            {"input_values": feature["input_values"]} for feature in features
        ]

        def onehot(lbl):
            onehot = [0] * 16
            onehot[int(lbl)] = 1
            return onehot

        output_features = [onehot(feature["labels"]) for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=True,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        # for val in batch['input_values']:
        #   print(val[:10])
        #   print(val[-10:])
        # print(batch['input_values'].shape)
        batch["labels"] = torch.tensor(output_features)
        # print(batch["labels"].argmax(-1))
        return batch

## Model

In [8]:
class Wav2Vec2ClassificationModel(Wav2Vec2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)

        self.wav2vec2 = Wav2Vec2Model(config)

        # self.inner_dim = 128
        self.feature_size = 999

        # self.tanh = nn.Tanh()
        # self.linear1 = nn.Linear(1024, self.inner_dim)
        # self.linear2 = nn.Linear(self.inner_dim * self.feature_size, 16)
        self.linear = nn.Linear(1024*self.feature_size, 16)
        self.init_weights()

    def freeze_feature_extractor(self):
        self.wav2vec2.feature_extractor._freeze_parameters()

    def forward(
        self,
        input_values,
        attention_mask=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
        labels=None,
    ):
        return_dict = (
            return_dict if return_dict is not None else self.config.use_return_dict
        )

        outputs = self.wav2vec2(
            input_values,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        x = outputs[0]
        # print(x.shape)
        # x = self.linear1(outputs[0])
        # x = self.tanh(x)
        # x = self.linear(x.view(-1, self.inner_dim * self.feature_size))
        # print(x.size(0))
        x = self.linear(x.view(-1,1024*self.feature_size))
        return {"logits": x}

# Getting the dataset

In [9]:
!cp -r /content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/dataset_wav.zip .

In [10]:
!cp /content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/metadata.xlsx .
!cp /content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/test_metadata.xlsx .

In [11]:
if os.path.exists('dataset_wav'):
  if len(os.listdir('dataset_wav')) == 0:
    os.system('unzip dataset_wav.zip')
else:
  os.system('unzip dataset_wav.zip')

In [12]:
metadata_path = '/content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/metadata.xlsx'
test_metadata_path = '/content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/test_metadata.xlsx'

In [13]:
dataset_wav_folder = 'dataset_wav'

In [14]:
def preprocess_metadata(metadata):
  metadata['Bahr'] = metadata['Bahr'].apply(lambda bahr: bahr.strip() if isinstance(bahr,str) else bahr)
  metadata['Clean Script'] = metadata['Script'].apply(lambda script: araby.strip_tashkeel(araby.strip_tatweel(script)))
  metadata['File Exist'] = metadata['Utterance name'].apply(lambda file_name: os.path.isfile(f'{dataset_wav_folder}/{file_name}'))
  metadata = metadata[metadata['File Exist'] == True]
  return metadata

In [15]:
metadata = preprocess_metadata(pd.read_excel(metadata_path))
test_metadata = preprocess_metadata(pd.read_excel(test_metadata_path))
train_metadata = metadata[[script not in test_metadata['No.'].values for script in metadata['No.']]]
train_metadata

,No.,Utterance name,Script,Bahr,Source,Name when extacted,Original name in document,Is_Malformed,Reciter_Label,Reciter_Gender,Further info on Bahr,Name of document,Clean Script,File Exist
0,1.0,P-1.aac,شتَّ شَعبُ الحيِّ بعدَ التئامْ وشجاكَ الر...,المديد,ديوان الطرماح ص227,1.aac,1.aac,0.0,0.0,0.0,NaN,بحر المديد,شت شعب الحي بعد التئام وشجاك الربع ربع ال...,True
1,2.0,P-2.aac,حَسَرَتْ عَنْهُ الرِّيَاحُ فَأَبْدَتْ من...,المديد,ديوان الطرماح ص227,2.aac,2.aac,0.0,0.0,0.0,NaN,NaN,حسرت عنه الرياح فأبدت منتأى كالقرو رهن ا...,True
2,3.0,P-3.aac,وخصيفَ اللَّونِ جادَتْ بهِ مَرْخَةٌ مِنْ ...,المديد,ديوان الطرماح ص227,3.aac,3.aac,0.0,0.0,0.0,NaN,NaN,وخصيف اللون جادت به مرخة من مخدج أو تمام,True
3,4.0,P-4.aac,بَيْنَ أظْآرٍ بِمَظْلُومَةٍ كَسَرَاةِ الس...,المديد,ديوان الطرماح ص227,4.aac,4.aac,0.0,0.0,0.0,NaN,NaN,بين أظآر بمظلومة كسراة الساق ساق الحمام,True
4,5.0,P-5.aac,مَنْزِلًا كَانَ لَنَا مَرَّةً وطنًا نحتلّ...,المديد,ديوان الطرماح ص227,5.aac,5.aac,0.0,0.0,0.0,NaN,NaN,منزلا كان لنا مرة وطنا نحتله كل عام,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3681,3731.0,P-3731.mp3,وَلِّ السَلاطينَ مَن تَوَلّاها وَاِلجَأ إ...,المنسرح,-,45.mp3,45.mp3,NaN,NaN,NaN,NaN,NaN,ول السلاطين من تولاها والجأ إليه تكن حدياها,True
3682,3732.0,P-3732.mp3,وَلا تَغُرَّنَّكَ الإِمارَةُ في غَيرِ أَم...,المنسرح,-,46.mp3,46.mp3,NaN,NaN,NaN,NaN,NaN,ولا تغرنك الإمارة في غير أمير وإن بها باهى,True
3683,3733.0,P-3733.mp3,فَإِنَّما المَلكُ رَبُّ مَملَكَةٍ قَد فَغ...,المنسرح,-,47.mp3,47.mp3,NaN,NaN,NaN,NaN,NaN,فإنما الملك رب مملكة قد فغم الخافقين سرياها,True
3684,3734.0,P-3734.mp3,مُبتَسِمٌ وَالوُجوهُ عابِسَةٌ سِلمُ العِد...,المنسرح,-,48.mp3,48.mp3,NaN,NaN,NaN,NaN,NaN,مبتسم والوجوه عابسة سلم العدى عنده كهيجاها,True


In [16]:
test_metadata

,No.,Utterance name,Script,Bahr,Source,Name when extacted,Original name in document,Further info on Bahr,Name of document,Clean Script,File Exist
0,89.0,P-89.aac,إنَّ بالشِّعبِ الَّذي دُونَ سَلْعٍ لَقتيل...,المديد,ديوان ثابت بن جابرص35,89.aac,89.aac,NaN,NaN,إن بالشعب الذي دون سلع لقتيلا دمه ما يطل,True
1,90.0,P-90.aac,خَلَّفَ الْعِبْءَ عَلَيَّ وَولَّى أنا بال...,المديد,ديوان ثابت بن جابرص35,90.aac,90.aac,NaN,NaN,خلف العبء علي وولى أنا بالعبء له مستقل,True
2,91.0,P-91.aac,ووراءَ الثَّأر منِّي ابنُ أختٍ مَصِعٌ عُق...,المديد,ديوان ثابت بن جابرص35,91.aac,91.aac,NaN,NaN,ووراء الثأر مني ابن أخت مصع عقدته ما تحل,True
3,92.0,P-92.aac,مُطْرِقٌ يَرْشَحُ سَمًّا كَمَا أَطْ رَقَ ...,المديد,ديوان ثابت بن جابرص35,92.aac,92.aac,NaN,NaN,مطرق يرشح سما كما أط رق أفعى ينفث السم صل,True
4,93.0,P-93.aac,خبرٌ ما نابَنا مُصْمَئِلٌّ جلَّ حتّى دقَّ...,المديد,ديوان ثابت بن جابرص35,93.aac,93.aac,NaN,NaN,خبر ما نابنا مصمئل جل حتى دق فيه الأجل,True
...,...,...,...,...,...,...,...,...,...,...,...
354,3682.0,P-3682.m4a,حِسانُ الوُجوهِ حِدادُ السُيو فِ يَبتَدِر...,المتقارب,ديوان قيس بن الخطيم ص 73,ÇáÊÓÌíá (98).m4a,التسجيل (98).m4a,NaN,NaN,حسان الوجوه حداد السيو ف يبتدر المجد شبانها,True
355,3683.0,P-3683.m4a,وَبِالشَوطِ مِن يَثرِبٍ أَعبُدٌ سَتَهلِكُ...,المتقارب,ديوان قيس بن الخطيم ص 73,ÇáÊÓÌíá (99).m4a,التسجيل (99).m4a,NaN,NaN,وبالشوط من يثرب أعبد ستهلك في الخمر أثمانها,True
356,3684.0,P-3684.m4a,يَهونُ عَلى الأَوسِ أَثمانُهُم إِذا راحَ ...,المتقارب,ديوان قيس بن الخطيم ص 73,ÇáÊÓÌíá (100).m4a,التسجيل (100).m4a,NaN,NaN,يهون على الأوس أثمانهم إذا راح يخطر نشوانها,True
357,3685.0,P-3685.m4a,أَتَتهُم عَرانينُ مِن مالِكٍ سِراعٌ إِلى ...,المتقارب,ديوان قيس بن الخطيم ص 73,ÇáÊÓÌíá (101).m4a,التسجيل (101).m4a,NaN,NaN,أتتهم عرانين من مالك سراع إلى الروع فتيانها,True


In [17]:
train_metadata,val_metadata = train_test_split(train_metadata, test_size=0.1,random_state=45)

# Training args

In [18]:
OUTPUT_PATH = '/content/drive/MyDrive/KFUPM-Master/ICS606/PostCourseWork/Models/EndToEndFineTunedSimpleDenseModel'
args_string=f"""
--model_name_or_path="/content/drive/MyDrive/KFUPM-Master/ICS606/Models/SinaiFineTuned/checkpoint-1600"
--output_dir={OUTPUT_PATH}
--overwrite_output_dir
--freeze_feature_extractor 
--num_train_epochs="5" 
--per_device_train_batch_size="2"
--learning_rate="3e-5" 
--warmup_steps="5" 
--evaluation_strategy="steps"
--save_steps="500" 
--eval_steps="250" 
--save_total_limit="1" 
--logging_steps="100"
--weight_decay=1e-1
--do_eval=1
--do_train=1
--preprocessing_num_workers=1
"""
def match_numeric_type(s):
  try:
    return json.loads(s)
  except:
    return s

# construct the args dict from the args string
args_dict = dict()
for line in args_string.splitlines():
  line = line.replace('--','').replace(r'\\','').replace('"','').replace("'",'').strip()
  if line:
    if '=' in line:
      arg_name,arg_value = line.split('=')
      args_dict[arg_name] = arg_value
    else:
      args_dict[line] = ''

# match args to dataclasses
args_class_attrs = defaultdict(list)
for arg_name,arg_value in args_dict.items():
  for ArgClass in (ModelArguments, DataTrainingArguments, TrainingArguments):
    if arg_name in ArgClass.__annotations__:
      args_class_attrs[ArgClass].append(arg_name)

# initialzie args objects
model_args = ModelArguments(**{arg_name:match_numeric_type(args_dict[arg_name]) for arg_name in args_class_attrs[ModelArguments]})
data_args = DataTrainingArguments(**{arg_name:match_numeric_type(args_dict[arg_name]) for arg_name in args_class_attrs[DataTrainingArguments]})
training_args = TrainingArguments(**{arg_name:match_numeric_type(args_dict[arg_name]) for arg_name in args_class_attrs[TrainingArguments]})

# Dataset Processing

In [19]:
train_dataset = Dataset.from_pandas(train_metadata)
eval_dataset = Dataset.from_pandas(val_metadata)
test_dataset = Dataset.from_pandas(test_metadata)

if data_args.max_train_samples is not None:
    train_dataset = train_dataset.select(range(data_args.max_train_samples))

if data_args.max_val_samples is not None:
    eval_dataset = eval_dataset.select(range(data_args.max_val_samples))

# Preprocessing the datasets.
# We need to read the aduio files as arrays and tokenize the targets.
resamplers = {  # The dataset contains all the uncommented sample rates
    48000: torchaudio.transforms.Resample(48000, 16000),
    44100: torchaudio.transforms.Resample(44100, 16000),
    32000: torchaudio.transforms.Resample(32000, 16000),
}

labels = {
    bahr: bahr_index
    for bahr_index, bahr in enumerate(sorted(set(train_metadata["Bahr"])))
}
print("labels are:", labels)
print("len of them is:", len(labels))

def speech_file_to_array_fn(batch):
    start = 0
    stop = 20
    srate = 16_000
    speech_array, sampling_rate = torchaudio.load(
        f'dataset_wav/{batch["Utterance name"]}'
    )
    speech_array = speech_array[0]
    batch["speech"] = resamplers[sampling_rate](speech_array).squeeze().numpy()
    batch["sampling_rate"] = srate
    batch["parent"] = labels[batch["Bahr"]]
    return batch


labels are: {'البسيط': 0, 'الخفيف': 1, 'الرجز': 2, 'الرمل': 3, 'السريع': 4, 'الطويل': 5, 'الكامل': 6, 'المتدارك': 7, 'المتقارب': 8, 'المجتث': 9, 'المديد': 10, 'المضارع': 11, 'المقتضب': 12, 'المنسرح': 13, 'الهزج': 14, 'الوافر': 15}
len of them is: 16


In [20]:
# for experimenting purposes
# train_dataset= Dataset.from_dict(train_dataset[:5])
# eval_dataset= Dataset.from_dict(eval_dataset[:5])
# test_dataset= Dataset.from_dict(test_dataset[:5])

In [21]:
print("processing the training set:")
train_dataset = train_dataset.map(
    speech_file_to_array_fn,
    remove_columns=train_dataset.column_names,
    num_proc=data_args.preprocessing_num_workers,
)

processing the training set:


  0%|          | 0/2978 [00:00<?, ?ex/s]

In [22]:
print("Process the eval set")
eval_dataset = eval_dataset.map(
    speech_file_to_array_fn,
    remove_columns=eval_dataset.column_names,
    num_proc=data_args.preprocessing_num_workers,
)

Process the eval set


  0%|          | 0/331 [00:00<?, ?ex/s]

In [23]:
print("process the test set")
test_dataset = test_dataset.map(
    speech_file_to_array_fn,
    remove_columns=test_dataset.column_names,
    num_proc=data_args.preprocessing_num_workers,

)

process the test set


  0%|          | 0/359 [00:00<?, ?ex/s]

In [24]:
feature_extractor = Wav2Vec2FeatureExtractor()
processor = CustomWav2Vec2Processor(feature_extractor=feature_extractor)

def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."
    batch["input_values"] = processor(
        batch["speech"],
        sampling_rate=batch["sampling_rate"][0],
    ).input_values
    batch["labels"] = batch["parent"]
    return batch


In [25]:
print("Process the trainset")
train_dataset = train_dataset.map(
    prepare_dataset,
    remove_columns=train_dataset.column_names,
    batch_size=training_args.per_device_train_batch_size,
    batched=True,
    num_proc=data_args.preprocessing_num_workers,
)

Process the trainset


  0%|          | 0/1489 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


In [26]:
print("Process the eval set")
eval_dataset = eval_dataset.map(
    prepare_dataset,
    remove_columns=eval_dataset.column_names,
    batch_size=training_args.per_device_train_batch_size,
    batched=True,
    num_proc=data_args.preprocessing_num_workers,
)

Process the eval set


  0%|          | 0/166 [00:00<?, ?ba/s]

In [27]:
print("Process the test set")
test_dataset = test_dataset.map(
    prepare_dataset,
    remove_columns=test_dataset.column_names,
    batch_size=training_args.per_device_train_batch_size,
    batched=True,
    num_proc=data_args.preprocessing_num_workers,
)

Process the test set


  0%|          | 0/180 [00:00<?, ?ba/s]

# Prepare the model for training

In [28]:
model = Wav2Vec2ClassificationModel.from_pretrained(
    "/content/drive/MyDrive/KFUPM-Master/ICS606/Models/SinaiFineTuned/checkpoint-1600",
    attention_dropout=0.01,
    hidden_dropout=0.01,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.01,
    # gradient_checkpointing=True,
    num_attention_heads=4,
)

if model_args.freeze_feature_extractor:
    model.freeze_feature_extractor()

def compute_metrics(pred):
    labels = pred.label_ids.argmax(-1)
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    report = classification_report(labels, preds)
    matrix = confusion_matrix(labels, preds)
    print(matrix)
    print('clasification report',report)
    print('accuracy',acc)
    return {"accuracy": acc}

# Data collator
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)
# Initialize our Trainer
trainer = CTCTrainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset if training_args.do_train else None,
    eval_dataset=eval_dataset if training_args.do_eval else None,
    tokenizer=processor.feature_extractor,
)

Some weights of the model checkpoint at /content/drive/MyDrive/KFUPM-Master/ICS606/Models/SinaiFineTuned/checkpoint-1600 were not used when initializing Wav2Vec2ClassificationModel: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2ClassificationModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ClassificationModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ClassificationModel were not initialized from the model checkpoint at /content/drive/MyDrive/KFUPM-Master/ICS606/Models/SinaiFineTuned/checkpoint-1600 and are newly initialized: ['linear.bias', 'linear.weight']
You should probably TRAIN this model on a down-stream task 

# Train the model

In [29]:
# Detecting last checkpoint.
last_checkpoint = None
if (
    os.path.isdir(training_args.output_dir)
    and training_args.do_train
    and not training_args.overwrite_output_dir
):
    last_checkpoint = get_last_checkpoint(training_args.output_dir)
    if last_checkpoint is None and len(os.listdir(training_args.output_dir)) > 0:
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. "
            "Use --overwrite_output_dir to overcome."
        )
    elif last_checkpoint is not None:
        print(
            f"Checkpoint detected, resuming training at {last_checkpoint}. To avoid this behavior, change "
            "the `--output_dir` or add `--overwrite_output_dir` to train from scratch."
        )

In [30]:
# Training
if training_args.do_train:
  if last_checkpoint is not None:
      checkpoint = last_checkpoint
  elif os.path.isdir(model_args.model_name_or_path):
      checkpoint = model_args.model_name_or_path
  else:
      checkpoint = None
  train_result = trainer.train(resume_from_checkpoint=checkpoint)
  trainer.save_model()

  # save the feature_extractor and the tokenizer
  if is_main_process(training_args.local_rank):
      processor.save_pretrained(training_args.output_dir)

  metrics = train_result.metrics
  max_train_samples = (
      data_args.max_train_samples
      if data_args.max_train_samples is not None
      else len(train_dataset)
  )
  metrics["train_samples"] = min(max_train_samples, len(train_dataset))

  trainer.log_metrics("train", metrics)
  trainer.save_metrics("train", metrics)
  trainer.save_state()

# Evaluation
results = {}
if training_args.do_eval:
  print("*** Evaluate ***")
  metrics = trainer.evaluate()
  max_val_samples = (
      data_args.max_val_samples
      if data_args.max_val_samples is not None
      else len(eval_dataset)
  )
  metrics["eval_samples"] = min(max_val_samples, len(eval_dataset))

  trainer.log_metrics("eval", metrics)
  trainer.save_metrics("eval", metrics)

Loading model from /content/drive/MyDrive/KFUPM-Master/ICS606/Models/SinaiFineTuned/checkpoint-1600.
You are resuming training from a checkpoint trained with 4.12.5 of Transformers but your current version is 4.23.1. This is not recommended and could yield to errors or unwanted behaviors.
There were missing keys in the checkpoint model loaded: ['linear.weight', 'linear.bias'].
There were unexpected keys in the checkpoint model loaded: ['lm_head.weight', 'lm_head.bias'].
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2978
  Num Epochs = 5
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  

Step,Training Loss,Validation Loss,Accuracy
250,5.758200,3.116628,0.395770
500,3.714400,3.281307,0.296073
750,2.579100,2.319489,0.504532
1000,2.155800,2.829971,0.392749
1250,1.928700,1.353338,0.655589
1500,1.485000,1.430462,0.688822
1750,0.757800,1.242043,0.722054
2000,0.672100,1.249228,0.740181
2250,0.683600,1.122461,0.770393
2500,0.658200,1.125319,0.785498


***** Running Evaluation *****
  Num examples = 331
  Batch size = 8


[[  0   0   0   0   0   9   1   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0  10   1   0   0   0   0   3   0   2   1   0]
 [  0   0   0   0   0   1   1   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   1  21   1   0   0   0   0   4   0   5   1   1]
 [  0   0   0   0   0   7   2   0   0   0   3   2   0   1   1   0]
 [  0   0   0   0   0 104   2   0   0   0   0   1   0   0   0   0]
 [  0   0   0   0   0  28   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   1   2   0   0   0   0   0   7   0   1   1   0]
 [  0   0   0   0   0   3   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   1   1   0   0   0   0   6   0   0   0   0]
 [  0   0   0   0   0   3   0   0   0   0   8   0   0   0   1   0]
 [  0   0   0   0   0   1   0   0   0   0   1  14   0   2   0   0]
 [  0   0   0   0   0   1   0   0   0   0   0   1   0   0   0   0]
 [  0   0   0   0   0  17   2   0   0   0   0   5   0   2   1   0]
 [  0   0   0   0   0   0   0   0   0   0   0   2   0   2   3 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Evaluation *****
  Num examples = 331
  Batch size = 

...

Saving model checkpoint to /content/drive/MyDrive/KFUPM-Master/ICS606/PostCourseWork/Models/EndToEndFineTunedSimpleDenseModel/checkpoint-7000
Configuration saved in /content/drive/MyDrive/KFUPM-Master/ICS606/PostCourseWork/Models/EndToEndFineTunedSimpleDenseModel/checkpoint-7000/config.json
Model weights saved in /content/drive/MyDrive/KFUPM-Master/ICS606/PostCourseWork/Models/EndToEndFineTunedSimpleDenseModel/checkpoint-7000/pytorch_model.bin
Feature extractor saved in /content/drive/MyDrive/KFUPM-Master/ICS606/PostCourseWork/Models/EndToEndFineTunedSimpleDenseModel/checkpoint-7000/preprocessor_config.json
Deleting older checkpoint [/content/drive/MyDrive/KFUPM-Master/ICS606/PostCourseWork/Models/EndToEndFineTunedSimpleDenseModel/checkpoint-6500] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 331
  Batch size = 8


[[  5   0   0   0   0   3   1   0   0   0   0   0   0   0   0   1]
 [  1  12   0   0   0   2   0   0   0   0   0   1   1   0   0   0]
 [  0   0   2   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0  32   0   0   0   0   0   0   0   0   0   1   1   0]
 [  0   0   0   2  13   0   0   0   0   0   0   0   0   1   0   0]
 [  2   0   0   1   0 104   0   0   0   0   0   0   0   0   0   0]
 [  0   2   0   0   0   4  22   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0  10   0   0   0   0   0   1   0   1]
 [  0   0   0   0   0   2   0   0   1   0   0   0   0   0   0   0]
 [  0   0   0   1   0   0   0   1   0   6   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0  12   0   0   0   0   0]
 [  0   0   0   0   0   0   0   1   0   0   0  17   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   2   0   0   0]
 [  0   0   0   0   0   1   0   0   0   0   0   0   0  26   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   7 



Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /content/drive/MyDrive/KFUPM-Master/ICS606/PostCourseWork/Models/EndToEndFineTunedSimpleDenseModel
Configuration saved in /content/drive/MyDrive/KFUPM-Master/ICS606/PostCourseWork/Models/EndToEndFineTunedSimpleDenseModel/config.json
Model weights saved in /content/drive/MyDrive/KFUPM-Master/ICS606/PostCourseWork/Models/EndToEndFineTunedSimpleDenseModel/pytorch_model.bin
Feature extractor saved in /content/drive/MyDrive/KFUPM-Master/ICS606/PostCourseWork/Models/EndToEndFineTunedSimpleDenseModel/preprocessor_config.json
Feature extractor saved in /content/drive/MyDrive/KFUPM-Master/ICS606/PostCourseWork/Models/EndToEndFineTunedSimpleDenseModel/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 331
  Batch size = 8


***** train metrics *****
  epoch                    =          5.0
  total_flos               = 8834475126GF
  train_loss               =       0.7908
  train_runtime            =   1:28:44.67
  train_samples            =         2978
  train_samples_per_second =        2.796
  train_steps_per_second   =        1.398
*** Evaluate ***


[[  5   0   0   0   0   3   1   0   0   0   0   0   0   0   0   1]
 [  1  12   0   0   0   2   0   0   0   0   0   1   1   0   0   0]
 [  0   0   2   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0  32   0   0   0   0   0   0   0   0   0   1   1   0]
 [  0   0   0   2  13   0   0   0   0   0   0   0   0   1   0   0]
 [  2   0   0   1   0 104   0   0   0   0   0   0   0   0   0   0]
 [  0   1   0   0   0   2  25   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0  10   0   0   0   0   0   1   0   1]
 [  0   0   0   0   0   2   0   0   1   0   0   0   0   0   0   0]
 [  0   0   0   1   0   0   0   1   0   6   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0  12   0   0   0   0   0]
 [  0   0   0   0   0   0   0   1   0   0   0  17   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   2   0   0   0]
 [  0   0   0   0   0   1   0   0   0   0   0   0   0  25   0   1]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   7 

# Evaluate the model

In [31]:
from sklearn.metrics import classification_report,confusion_matrix
preds = trainer.predict(test_dataset=test_dataset)
preds

***** Running Prediction *****
  Num examples = 359
  Batch size = 8


[[ 0  0  0  0  0 23  2  0  0  0  0  0  0  0  0  0]
 [ 0  5  0  0  0  0 12  0  0  0  0  0  0  4  0  0]
 [ 0  3  0  5  1  0  0  3  0  2  0  0  0  1  1  0]
 [ 2  0  0  0  0 11  1  0  0  0  0  0  0  1  0 15]
 [ 1  0  0  0  0 13  1  0  0  0  0  0  0  0  0  5]
 [ 0  0  0  0  0 35  0  0  0  0  0  0  0  0  0  0]
 [ 1  1  0  0  1  2 22  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 17  0  0  0  0  0  0  1  0]
 [ 3  0  0  0  0 15  0  0  0  0  0  0  0  0  0  1]
 [ 0  0  0  0  0  0  0  0  0 19  0  0  0  2  0  0]
 [ 0 24  0  0  0  0  0  0  0  0  0  0  1  3  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 18  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 15  0  0  0]
 [ 1  0  0  0  0 27  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  0  4  0  1  0  2  0  0  9  0]
 [ 0  0  0  0  0 18  2  0  0  0  0  0  0  0  0  1]]
clasification report               precision    recall  f1-score   support

           0       0.00      0.00      0.00        25
           1       0.15      0.24      0.19        21

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


PredictionOutput(predictions=array([[ -3.338646  ,  16.463861  , -13.128032  , ...,  11.298483  ,
         -1.6219057 ,  -6.781573  ],
       [  0.16977693,  15.073946  ,  -4.667925  , ...,   9.75125   ,
          6.5783153 ,  -6.055086  ],
       [ -4.6537924 ,  10.22328   ,  -9.918781  , ...,  20.870173  ,
          2.6082003 ,  -7.810011  ],
       ...,
       [ 17.478794  ,   5.366128  , -13.650666  , ...,  11.434692  ,
        -17.961357  ,   0.4294264 ],
       [  4.965478  ,   2.933916  ,  -9.793499  , ...,   0.3540962 ,
         -1.261696  ,   1.9555149 ],
       [  3.9658484 ,   0.585757  ,  -0.80362606, ...,   3.9697907 ,
        -10.728446  ,  -0.07062101]], dtype=float32), label_ids=array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), metrics={'test_loss': 10.130943298339844, 'test_accuracy': 0.39275766016713093, 'test_runt

In [32]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive
